In [3]:
"""
Created on Mon February 22nd, 2022
@author: Eleftheria Chatzitheodoridou

Description: 
This script is designed to read NIfTI files that contain Grade 2, 3 (LGG) tumors from the
local directory, extract the areas where the tumor is present across (x,y,z) by finding
the (min, max) of each axis, then normalize the intensity of the extracted 2D images and
save the slices on a local directory as .png files.

Note: This script was created on a Linux computer and some commands may not work on Windows/MAC OS.
"""

import os.path 
import numpy as np
import glob
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
import re
from PIL import Image
from numpy import ndarray

# Folder where the created images will be saved in
out_path = r'/local/data1/elech646/Tumor_grade_classification/Slices'

# Create subfolders
if not os.path.exists(out_path + "/sagittal_grade_classification"):
    os.mkdir(out_path + "/sagittal_grade_classification")
    
if not os.path.exists(out_path + "/frontal_grade_classification"):
    os.mkdir(out_path + "/frontal_grade_classification")

if not os.path.exists(out_path + "/trans_grade_classification"):
    os.mkdir(out_path + "/trans_grade_classification")

# Add LGG path
sag_path = out_path + "/sagittal_grade_classification" + "/LGG"
fro_path = out_path + "/frontal_grade_classification" + "/LGG"
tra_path = out_path + "/trans_grade_classification" + "/LGG"

if not os.path.exists(sag_path):
    os.mkdir(sag_path)
    
if not os.path.exists(fro_path):
    os.mkdir(fro_path)
    
if not os.path.exists(tra_path):
    os.mkdir(tra_path)

filepath = []
# Scanning files + directories
for roots, dirs, files in os.walk("/local/data1/elech646/Tumor_grade_classification/LGG"):
    for name in files:
        if name.endswith((".nii.gz",".nii")):
                    filepath.append(roots + os.path.sep + name)
                
# Creating the images
for i, file in enumerate(filepath):
    img = nib.load(file)
    img_data = img.get_fdata()  # Getting data matrix
    patient = file.split("LGG/")[1].split("/")[0] + "/"
    if not os.path.exists(sag_path + "/" + patient):
        os.mkdir(sag_path + "/" + patient)
    if not os.path.exists(fro_path + "/" + patient):
        os.mkdir(fro_path + "/" + patient)
    if not os.path.exists(tra_path + "/" + patient):
        os.mkdir(tra_path + "/" + patient)
    
    # Maximum and minimum slices in which the tumor is present
    tr_0 = min(ndarray.nonzero(img_data)[0])    # xmin
    tr_1 = max(ndarray.nonzero(img_data)[0])    # xmax
    fr_0 = min(ndarray.nonzero(img_data)[1])    # ymin
    fr_1 = max(ndarray.nonzero(img_data)[1])    # ymax
    sag_0 = min(ndarray.nonzero(img_data)[2])   # zmin
    sag_1 = max(ndarray.nonzero(img_data)[2])   # zmax
    
    # Creating the images in the Sagittal Plane (yz)
    img_sag = np.rot90(img_data, axes = (1, 2)) # yz plane sagittal
    img_sag = np.flip(img_sag, 0) # flip the image left/right since mango was used
    
    for sag in range(sag_0, sag_1 + 1):
        perc = int(((sag - sag_0)/(sag_1 - sag_0))*100) # Percentage along the selected slices
        tmp = img_sag[sag,:,:]
        min_v = img_sag.min()
        max_v = img_sag.max()
        
        # Normalize image
        tmp_norm = (255*(tmp - min_v) / max_v).astype(np.uint8) 
        
        # Add modality        
        modality = filepath[i].split("_",1)[1].split(".nii", 1)[0].split("_")[-1]
        
        # Name the files
        title = sag_path + "/" + patient + patient[:-1] + '_' + modality + str(perc)
        
        # Convert to RGB
        im = Image.fromarray(tmp_norm).convert('RGB')
        
        # Create output path
        out_path_f = out_path + "/" 
                
        # Save images
        im.save(os.path.join(out_path_f, title + ".png"))
        
    # Creating the images in the Frontal/Coronal Plane (xz)
    img_fr = np.rot90(img_data, axes = (0,2)) # xz plane frontal 
    for front in range(fr_0, fr_1 + 1):
        perc = int(((front - fr_0) /(fr_1 - fr_0))*100) # Percentage along the selected slices
        tmp = img_fr[:,front,:]
        min_v = img_fr.min()
        max_v = img_fr.max()
        
        # Normalize image
        tmp_norm = (255*(tmp - min_v) / max_v).astype(np.uint8) 
        
        # Add modality        
        modality = filepath[i].split("_",1)[1].split(".nii", 1)[0].split("_")[-1]
        
        # Name the files
        title = fro_path + "/" + patient + patient[:-1] + '_' + modality + str(perc)
        
        # Convert to RGB
        im = Image.fromarray(tmp_norm).convert('RGB')
        
        # Create output file path
        out_path_f = out_path + "/" 
        
        # Save images
        im.save(os.path.join(out_path_f, title + ".png"))
    
    # Creating the images in the Transversal/Axial Plane (xy)
    img_tr = np.rot90(img_data, 3, axes = (0,1)) # xy plane transversal
    x,y,z = img_tr.shape
    
    # Inverting slices upside/down since mango was used
    tr_0 = z-max(ndarray.nonzero(img_tr)[0])
    tr_1 = z-min(ndarray.nonzero(img_tr)[0])
    
    for transv in range(tr_0, tr_1):
        perc = int(((transv - tr_0) / (tr_1 - tr_0))*100) # Percentage along the selected slices
        tmp = img_tr[:,:,transv]
        min_v = img_tr.min()
        max_v = img_tr.max()
        
        # Normalize image
        tmp_norm = (255*(tmp - min_v) / max_v).astype(np.uint8) 
        
        # Add modality        
        modality = filepath[i].split("_",1)[1].split(".nii", 1)[0].split("_")[-1]
        
        # Name the files
        title = tra_path + "/" + patient + patient[:-1] + '_' + modality + str(perc)
        
        # Convert to RGB
        im = Image.fromarray(tmp_norm).convert('RGB')
        
        # Create output file path
        out_path_f = out_path + "/" 
        
        # Save images
        im.save(os.path.join(out_path_f, title + ".png"))